In [8]:
# importing libraries 
import pandas as pd
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import itertools
from torch.utils.data import TensorDataset
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import os
from datasets import Dataset
# import packages (remember to import the local_transoformers package in your folder correctly!)
from local_transformers_bioClinical_BERT.src.transformers import (
    TrainingArguments,
    Trainer
) 
import pandas as pd
import torch
import random
from datasets import Dataset
from torch.utils.data import DataLoader
import itertools
from torch.utils.data import TensorDataset
import tensorflow as tf
from sklearn.metrics import make_scorer, average_precision_score
from local_transformers_bioClinical_BERT.src.transformers.models.CustombioClinicalBERT import CustomBioClinicalBertForCombinedLearning # reads the local_transformers folder instead of the actual transformers package
from local_transformers_bioClinical_BERT.src.transformers.CustomComputeLoss import CustomTrainer # reads the local_transformers folder instead of the actual transformers package
from transformers import AutoModel
import os
import pickle
from local_transformers_bioClinical_BERT.src.transformers.trainer_callback import TrainerCallback
import matplotlib.pyplot as plt
import math
import numpy as np
from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import RepeatedKFold, cross_val_predict, cross_val_score
import numpy as np
from numpy import mean, std
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
from tqdm import tqdm
import statistics
import numpy
from local_transformers_bioClinical_BERT.src.transformers.data.data_collator import DataCollatorForLanguageModeling
from local_transformers_bioClinical_BERT.src.transformers.models.auto.tokenization_auto import AutoTokenizer
from sklearn.model_selection import KFold
if tf.test.is_gpu_available():
    print('GPU is available')
else:
    print('GPU is not available, running on CPU')

# reading the data
df=pd.read_csv("df.csv")
df["death_in_30"] = df["death_in_30"].astype(int)
df["DVT"] = df["DVT"].astype(int)
df["PE"] = df["PE"].astype(int)
df["PNA"]=df["PNA"].astype(int)
df["post_aki_status"]=df["post_aki_status"].astype(int)
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = CustomBioClinicalBertForCombinedLearning.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",output_hidden_states=True,weights=None,lambda_constant=2) #NOTE: Change lambda constant to the appropriate one


# preparing the clinical notes for training and fine-tuning
# this function here helps set up and prepare the data for fine-tuning, including those of the additional labels. 
def tokenize_and_prepare(batch,text_col,outcome_col):
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    encodings = tokenizer(batch[text_col], padding=True, truncation=True, max_length=164) #note: original 164
    additional_labels = torch.tensor(batch[outcome_col], dtype=torch.float)  # note this line ensures that the additional labels are considered during semi-supervised fine-tuning 
    return {"input_ids": encodings["input_ids"], "attention_mask": encodings["attention_mask"], "additional_labels": additional_labels, "labels": encodings["input_ids"]}

# preparing the data for fine-tuning
def prepare_data(df, text_col, outcome_col):
    texts = list(df[text_col])
    outcomes = list(df[outcome_col])
    data = {text_col: texts, outcome_col: outcomes}
    dataset = Dataset.from_dict(data)
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    dataset = dataset.map(lambda batch: tokenize_and_prepare(batch, text_col, outcome_col), batched=True, batch_size=len(texts))
    dataset.set_format("torch", columns=["input_ids", "attention_mask", "additional_labels", "labels"])
    return dataset

GPU is available


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing CustomBioClinicalBertForCombinedLearning: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBioClinicalBertForCombinedLearning were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['auxiliary.weight', 'auxiliary.bias']
You should probably TRAIN this model on a down-stream task to be able to 

In [2]:
len(df)

84875

In [9]:
# this function initiates the fine-tuning parameters and fine-tunes the model
def get_model(name, train_dataset,val_dataset):
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    model = CustomBioClinicalBertForCombinedLearning.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",output_hidden_states=True,weights=None,lambda_constant=2) #NOTE: Change lambda constant to the appropriate one
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=5,
        per_device_train_batch_size=36,
        per_device_eval_batch_size=36,
        warmup_steps=1500, 
        weight_decay=0.001,
        learning_rate=0.00001, 
        logging_dir=None,
        logging_steps=1000,
        save_strategy="epoch")#    
    
    # Define the data collator for masked language modeling
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
    # Instantiate the Trainer
    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer)
    trainer.train()
    trainer.model.save_pretrained(name)
    trainer.tokenizer.save_pretrained(name)

In [10]:
def get_bioclinicalBERT_updated(texts, name):
    notes = list(texts)
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(name)
    model = CustomBioClinicalBertForCombinedLearning.from_pretrained(name, output_hidden_states=True)

    # Check if CUDA is available and if so, move the model to GPU
    if torch.cuda.is_available():
        model = model.to("cuda")

    # Tokenize texts and create input IDs and attention masks
    input_ids, attention_masks = [], []
    for text in notes:
        encoded_input = tokenizer.encode_plus(text, max_length=128, padding='max_length', truncation=True,
                                              return_attention_mask=True, return_tensors='pt')  # change max_length if needed [NOTE: max_length may differ across task]
        input_ids.append(encoded_input['input_ids'])
        attention_masks.append(encoded_input['attention_mask'])

    # Convert lists to tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    # If CUDA is available, move the tensors to GPU
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")
        attention_masks = attention_masks.to("cuda")

    # Batch size for processing
    batch_size = 32
    # Initialize an empty list to store embeddings
    embeddings = []
    # Process input IDs and attention masks in batches
    for i in tqdm(range(0, len(input_ids), batch_size)):
        input_ids_batch = input_ids[i:i+batch_size]
        attention_masks_batch = attention_masks[i:i+batch_size]

        with torch.no_grad():
            # Get the model's output
            output = model(input_ids=input_ids_batch, attention_mask=attention_masks_batch)

        # Retrieve the last_hidden_state from the output
        last_hidden_state = output.hidden_states[-1]

        # Optionally, you can obtain the pooled_output by averaging or max-pooling the last_hidden_state
        pooled_output = torch.mean(last_hidden_state, dim=1)

        # If CUDA is available, move the embeddings back to CPU
        if torch.cuda.is_available():
            pooled_output = pooled_output.to("cpu")

        # Add the embeddings to the list
        embeddings.extend(pooled_output.numpy())

    embeddings = np.array(embeddings)
    return embeddings


In [11]:
def K_fold_val(outcome_col, df):
    # Initialize the StratifiedKFold class
    skf = KFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize lists to hold scores
    auroc_scores = []
    auprc_scores = []


    # defining the XGboost parameters to test out for CV
    param_grid = {
        'learning_rate': [0.1,0.15,0.3],
        'max_depth': [4,5,6,7,8],
        'min_child_weight':[1,2,4]
        }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df,df["PE"])):
        # Split the data into train/test sets
        # Create a new column for embeddings based on the training set only
        name=f"{outcome_col}_fold_{i}"
        i=i+1
        # print(name)

        train, test = df.iloc[train_index], df.iloc[test_index]
        train_new=((train.sample(frac=7/8,random_state=42)))
        val_new = train[~train.index.isin(train_new.index)]
        train_notes=(train_new.reset_index(drop=True))
        val_notes=(val_new.reset_index(drop=True))
        train_dataset = prepare_data(train_notes,"AN_PROC_NAME", outcome_col)
        val_dataset = prepare_data(val_notes,"AN_PROC_NAME", outcome_col)s



        if os.path.isdir(name):
            print("passed")
        else:
            train_new=((train.sample(frac=7/8,random_state=42)))
            val_new = train[~train.index.isin(train_new.index)]
            train_notes=(train_new.reset_index(drop=True))
            val_notes=(val_new.reset_index(drop=True))
            train_dataset = prepare_data(train_notes,"AN_PROC_NAME", outcome_col)
            val_dataset = prepare_data(val_notes,"AN_PROC_NAME", outcome_col)
            get_model(name, train_dataset,val_dataset)

        if os.path.isfile(f"{name}.pickle"):
            with open(f"{name}.pickle", 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_bioclinicalBERT_updated(train["AN_PROC_NAME"], name)
            X_test = get_bioclinicalBERT_updated(test["AN_PROC_NAME"], name)
            with open(f"{name}.pickle", 'wb') as f:
                pickle.dump([X_train,X_test], f) 
       

        # Prepare training and testing data
        y_train = train[outcome_col]
        y_test = test[outcome_col]

        # run the XGBoost classifier
        model = xgb.XGBClassifier(random_state=42,tree_method='gpu_hist')
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision', verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]

        # Compute the AUROC and AUPRC
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        with open(f"update_{outcome_col}.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores}, f)

    # Now you can calculate the mean and standard deviation
    mean_auroc = np.mean(auroc_scores)
    std_auroc = np.std(auroc_scores)
    mean_auprc = np.mean(auprc_scores)
    std_auprc = np.std(auprc_scores)
    print(mean_auroc)
    print(mean_auprc)
    
    return(f"metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}")


In [6]:
results=K_fold_val("death_in_30", df) # note: lambda = 2

0it [00:00, ?it/s]Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing CustomBioClinicalBertForCombinedLearning: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBioClinicalBertForCombinedLearning were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['auxiliary.bias', 'auxiliary.weight']
You should probably TRAIN this model on a down-stream t

test


Step,Training Loss
1000,2.113700
2000,1.117100
3000,0.911300
4000,0.803300
5000,0.736600
6000,0.717700
7000,0.686700
8000,0.665600


100%|██████████| 531/531 [00:36<00:00, 14.72it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_c

...

Step,Training Loss
1000,2.076500
2000,1.138500
3000,0.925400
4000,0.800500
5000,0.755300
6000,0.727300
7000,0.690200
8000,0.676000


100%|██████████| 531/531 [00:36<00:00, 14.73it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [4:09:32, 2994.50s/it]

0.8681895091625602
0.1786402682204727


In [7]:
print(results) 

metrics: Mean AUROC: 0.8681895091625602, SD AUROC: 0.0037008158245333865, Mean AUPRC: 0.1786402682204727, SD AUPRC: 0.01547084476486198


## Part II: DVT

In [8]:
results=K_fold_val("DVT", df) 

0it [00:00, ?it/s]Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing CustomBioClinicalBertForCombinedLearning: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBioClinicalBertForCombinedLearning were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['auxiliary.bias', 'auxiliary.weight']
You should probably TRAIN this model on a down-stream t

test


Step,Training Loss
1000,1.998000
2000,1.020800
3000,0.811200
4000,0.711700
5000,0.647300
6000,0.625400
7000,0.600500
8000,0.583600


100%|██████████| 531/531 [00:35<00:00, 14.80it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_c

...

Step,Training Loss
1000,1.988400
2000,1.031000
3000,0.820700
4000,0.705700
5000,0.660100
6000,0.628500
7000,0.602800
8000,0.583100


100%|██████████| 531/531 [00:36<00:00, 14.73it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [4:04:33, 2934.68s/it]

0.7764935457929723
0.024368040002732103


In [9]:
print(results) 

# training_args = TrainingArguments(
#     output_dir="./results",
#     num_train_epochs=5,
#     per_device_train_batch_size=36,
#     per_device_eval_batch_size=36,
#     warmup_steps=1500, 
#     weight_decay=0.001,
#     learning_rate=0.00001, 
#     logging_dir=None,
#     logging_steps=1000,
#     save_strategy="epoch")
#   lambda = 2

metrics: Mean AUROC: 0.7764935457929723, SD AUROC: 0.013803181074430647, Mean AUPRC: 0.024368040002732103, SD AUPRC: 0.003358866676253976


## Part III: PE

In [5]:
results=K_fold_val("PE", df) 

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   8.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   8.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   8.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   8.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   8.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   8.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   8.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   8.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   8.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   8.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   8.2s
[CV] END .learning_rate=0.1, max_depth=4

1it [10:40, 640.66s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4

2it [18:04, 524.61s/it]

...

Step,Training Loss
1000,1.548500
2000,0.849200
3000,0.693400
4000,0.629900
5000,0.585200
6000,0.539700
7000,0.530500
8000,0.500300
9000,0.489800
10000,0.471400


100%|██████████| 531/531 [00:41<00:00, 12.64it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [3:43:18, 2679.68s/it]

0.7279921765082444
0.013750461451064715


In [6]:
print(results) 
# trend so far: higher lambda goes, auroc increases, but auprc drops. 

    # training_args = TrainingArguments(
    #     output_dir="./results",
    #     num_train_epochs=8,
    #     per_device_train_batch_size=36,
    #     per_device_eval_batch_size=36,
    #     warmup_steps=500,
    #     weight_decay=0.00001,
    #     learning_rate=0.00001,
    #     logging_dir="./logs",
    #     logging_steps=1000,
    #     save_strategy="epoch") max_length=296
    # lambda=1

metrics: Mean AUROC: 0.7279921765082444, SD AUROC: 0.01849155809782009, Mean AUPRC: 0.013750461451064715, SD AUPRC: 0.0027140819711502302


## Part 4: PNA

In [6]:
results=K_fold_val("PNA", df) 

0it [00:00, ?it/s]Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing CustomBioClinicalBertForCombinedLearning: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBioClinicalBertForCombinedLearning were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['auxiliary.weight', 'auxiliary.bias']
You should probably TRAIN this model on a down-stream t

test


Step,Training Loss
1000,1.517700
2000,0.852800
3000,0.733200
4000,0.656000
5000,0.632600
6000,0.595000


100%|██████████| 531/531 [01:13<00:00,  7.18it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   6.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   6.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   6.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   6.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   6.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   6.5s
[CV] END .learning_rate=0.1, max_depth=4, min_c

...

Step,Training Loss
1000,1.507200
2000,0.856100
3000,0.713600
4000,0.660300
5000,0.624600
6000,0.592400


100%|██████████| 531/531 [01:13<00:00,  7.20it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   6.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   6.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   6.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   6.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   6.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   6.8s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [7:51:56, 5663.29s/it]

0.8222390851175219
0.04277720754878464


In [7]:
print(results) 
#     training_args = TrainingArguments(
#         output_dir="./results",
#         num_train_epochs=8,
#         per_device_train_batch_size=36,
#         per_device_eval_batch_size=36,
#         warmup_steps=1500,
#         weight_decay=0.00001,
#         learning_rate=0.00001,
#         logging_dir="./logs",
#         logging_steps=1000,
#         save_strategy="epoch")
# lambda =1

metrics: Mean AUROC: 0.8222390851175219, SD AUROC: 0.0120772936982821, Mean AUPRC: 0.04277720754878464, SD AUPRC: 0.008245197571789677


## Part 5: post_AKI_status

In [14]:
results=K_fold_val("post_aki_status", df) # note: lambda = 2

0it [00:00, ?it/s]Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing CustomBioClinicalBertForCombinedLearning: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBioClinicalBertForCombinedLearning were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['auxiliary.bias', 'auxiliary.weight']
You should probably TRAIN this model on a down-stream t

test


Step,Training Loss
1000,2.509400
2000,1.599600
3000,1.363600
4000,1.258100
5000,1.203500
6000,1.183500
7000,1.148500
8000,1.118500


100%|██████████| 531/531 [00:35<00:00, 14.87it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_c

...

Step,Training Loss
1000,2.514600
2000,1.601700
3000,1.379900
4000,1.250100
5000,1.203100
6000,1.181200
7000,1.121000
8000,1.116200


100%|██████████| 531/531 [00:35<00:00, 14.76it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [4:12:22, 3028.59s/it]

0.8426139984308101
0.4949280648252817


In [15]:
print(results) 

# training_args = TrainingArguments(
#     output_dir="./results",
#     num_train_epochs=5,
#     per_device_train_batch_size=36,
#     per_device_eval_batch_size=36,
#     warmup_steps=1500, 
#     weight_decay=0.001,
#     learning_rate=0.00001, 
#     logging_dir=None,
#     logging_steps=1000,
#     save_strategy="epoch")
# lambda=2
# max length = 164  

metrics: Mean AUROC: 0.8426139984308101, SD AUROC: 0.00578335107544615, Mean AUPRC: 0.4949280648252817, SD AUPRC: 0.015067105720494384


## delirium

In [16]:
df = df.dropna(subset=['postop_del']).reset_index(drop=True)
df["postop_del"]=df["postop_del"].astype(int)
results=K_fold_val("postop_del", df) # note: lambda = 2


0it [00:00, ?it/s]

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing CustomBioClinicalBertForCombinedLearning: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBioClinicalBertForCombinedLearning were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['auxiliary.bias', 'auxiliary.weight']
You should probably TRAIN this model on a down-stream task to be able to 

test


Step,Training Loss
1000,2.861400


100%|██████████| 77/77 [00:05<00:00, 14.72it/s]


...

Step,Training Loss
1000,2.875200


100%|██████████| 77/77 [00:05<00:00, 15.00it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [46:45, 561.12s/it]

0.6913579429420726
0.6626036937991518


In [17]:
print(results) 
    # parameters used:
    # training_args = TrainingArguments(
    #     output_dir="./results",
    #     num_train_epochs=5,
    #     per_device_train_batch_size=36,
    #     per_device_eval_batch_size=36,
    #     warmup_steps=1500, 
    #     weight_decay=0.001,
    #     learning_rate=0.00001, 
    #     logging_dir=None,
    #     logging_steps=1000,
    #     save_strategy="epoch")
    # lambda=2, max length=164

metrics: Mean AUROC: 0.6913579429420726, SD AUROC: 0.006062653040524244, Mean AUPRC: 0.6626036937991518, SD AUPRC: 0.008247504863644348
